In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import sys
sys.path.append("/home/scottmoore2_snhu/Desktop")
# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import dash

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



#Import AnimalShelter object from animal_shelter class
from Animal_Shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()


app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px', 'display': 'block', 'margin': 'auto'}),
    
    html.Center(html.B(html.H1('Scott Moore | Last Updated: February 23, 2025'))),
    html.Hr(),
    
    #Filter buttons for rescue type
    html.Div(className='buttonRow',
             style={'display' : 'flex', 'justify-content': 'left'},
             children=[
                 html.Button(id='water-button', n_clicks=0, children='Water Rescue'),
                 html.Button(id='mountain-button', n_clicks=0, children='Mountain and Wilderness Rescue'),
                 html.Button(id='disaster-button', n_clicks=0, children='Disaster and Tracking'),
                 html.Button(id='reset-button', n_clicks=0, children='Reset')
             ]),
        
    html.Hr(),
    
    #Data Table
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         row_selectable="single",
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),

#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('water-button', 'n_clicks'),
               Input('mountain-button', 'n_clicks'),
               Input('disaster-button', 'n_clicks'),
               Input('reset-button', 'n_clicks')]
             )
def update_table(water_button, mountain_button, disaster_button, reset_button):
    query = {}
    ctx = dash.callback_context 
    if not ctx or not ctx.triggered:
        return df.to_dict('records')
    
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
        
        #Rescue type filtering
    if trigger_id == 'water-button':
        query["breed"]={"$in": ["Labrador Retriever", "Newfoundland", "Chesapeake Bay Retriever"]}
        query["sex_upon_outcome"] = "Intact Female"
        query["age_upon_outcome_in_weeks"] = {"$gte": 26, "$lte": 156}
        
    elif trigger_id == 'mountain-button':
        query["breed"]={"$in": ["German Shepherd", "Alaskan Malamute", "Bernese Mountain Dog"]}
        query["sex_upon_outcome"] = "Intact Male"
        query["age_upon_outcome_in_weeks"] = {"$gte": 26, "$lte": 156}
        
    elif trigger_id == 'disaster-button':
        query["breed"]={"$in": ["Bloodhound", "Doberman Pinscher", "Rottweiler"]}
        query["sex_upon_outcome"] = "Intact Male"
        query["age_upon_outcome_in_weeks"] = {"$gte": 20, "$lte": 300}
            
            
    df_filtered = pd.DataFrame.from_records(db.read(query))
    
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
        
    return df_filtered.to_dict('records')

#Pie Chart callback
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_pie_chart(data):
    df_filtered = pd.DataFrame.from_dict(data)
    
#if there are no results, display "No Data" in red
    if df_filtered.empty:
        return [html.P("No Data", style={"color": "red", "text-align": "center"})]
    
    return[
        dcc.Graph(
            figure=px.pie(df_filtered, names="breed", title="Breed Distribution")
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:27267/
